In [ ]:
# ! pip install -qU \
#     openai \
#     pinecone-client \
#     langchain \
#     tiktoken \
#     dotenv

In [18]:
from dotenv import dotenv_values

config = dotenv_values("../.env")

## Initializing Embedding Model and Vector DB

In [32]:
from FlagEmbedding import FlagModel

model = FlagModel('BAAI/bge-large-en-v1.5', 
                query_instruction_for_retrieval="Represent this sentence for searching relevant passages: ",
                use_fp16=True)

embed = lambda x: model.encode(x).tolist()            

In [33]:
PINECONE_API = config['PINECONE_API']
PINECONE_ENV = config['PINECONE_ENV']

In [34]:
import pinecone

# find API key in console at app.pinecone.io
YOUR_API_KEY = PINECONE_API
# find ENV (cloud region) next to API key in console
YOUR_ENV = PINECONE_ENV

index_name = 'llm-recommender-system'
pinecone.init(
    api_key=YOUR_API_KEY,
    environment=YOUR_ENV
)

if index_name not in pinecone.list_indexes():
    # we create a new index
    pinecone.create_index(
        name=index_name,
        metric='dotproduct',
        dimension=1536  # 1536 dim of text-embedding-ada-002
    )

In [5]:
index = pinecone.Index(index_name)
index_stats_response = index.describe_index_stats()

print(index_stats_response)

{'dimension': 1024,
 'index_fullness': 0.01,
 'namespaces': {'': {'vector_count': 1000}},
 'total_vector_count': 1000}


## Indexing Embedding Vectors

In [6]:
import pandas as pd

In [9]:
sample_books = pd.read_csv("../data/sample_books.csv")

In [10]:
sample_books

,Id,Name,Description
0,1374307,"Tough Times Never Last, but Tough People Do!","Name your problem, and you name your possibili..."
1,1334737,Black Gold,This bracing novel introduces vivid African-Am...
2,1598771,The Autobiography of Lincoln Steffens,Rediscover a man Americans turned to not only ...
3,2179832,Bright Shiny Morning,One of the most celebrated and controversial a...
4,4421731,The Socially Skilled Child Molester: Different...,Know what signs indicate a child molester!<br ...
...,...,...,...
95,2442839,"National Portrait Gallery, London",-- The first full catalogue of this major art ...
96,2787050,Collecting Toy Cars and Trucks: A Collector's ...,This indispensable identification and value gu...
97,4506874,Self Related Cognitions In Anxiety And Motivation,First published in 1986. Routledge is an impri...
98,3002398,Predators: A Pop-up Book with Revolutionary Te...,Get up close and personal with some of the wor...


In [11]:
sample_books['category'] =  sample_books.apply(lambda x: 'popular' if x.name < 50 else 'recommended', axis=1)

In [12]:
sample_books['user_id'] = sample_books.apply(lambda x: x.name % 10 + 1 if x['category'] == 'recommended' else None, axis = 1)

In [13]:
sample_books['ranking'] = sample_books.groupby('user_id').cumcount() + 1

In [14]:
sample_books.rename(columns={'Id' : 'id', 'Name': 'title', 'Description': 'description'}, inplace=True)

In [94]:
data = sample_books

In [96]:
data[data['user_id'] == 1]

,id,title,description,category,user_id,ranking
50,4589735,Informal Economy: A Research G,First published in 1991. Routledge is an impri...,recommended,1.0,1.0
60,1112628,In Amma's Healing Room: Gender and Vernacular ...,"""[I]t is extremely salubrious to see the ways ...",recommended,1.0,2.0
70,3041182,Vision & Prophecy in Amos Revised,In 1955 John D. W. Watts presented the faculty...,recommended,1.0,3.0
80,2243440,Environmental Regulation: Statutory Supplement...,This excellent resource takes full advantage o...,recommended,1.0,4.0
90,4502407,Hooded Americanism: The History of the Ku Klux...,"""The only work that treats Ku Kluxism for the ...",recommended,1.0,5.0


In [105]:
from tqdm.auto import tqdm
from uuid import uuid4

batch_size = 100

texts = []
metadatas = []

for i in tqdm(range(0, len(data), batch_size)):
    # get end of batch
    i_end = min(len(data), i+batch_size)
    batch = data.iloc[i:i_end]
    # first get metadata fields for this record
    metadatas = [
    {
        'category': record['category'],
        'title': record['title'],
        'description': record['description'],
        **({'user_id': str(int(record['user_id']))} if record['category'] == 'recommended' else {})
    }
    for _, record in batch.iterrows()]
    # get the list of contexts / documents
    documents = batch['description']
    # create document embeddings
    embeds = embed.embed_documents(documents)
    # get IDs
    ids = batch['id'].astype(str)
    # add everything to pinecone
    index.upsert(vectors=zip(ids, embeds, metadatas))

100%|██████████| 1/1 [00:04<00:00,  4.98s/it]


In [106]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.001,
 'namespaces': {'': {'vector_count': 100}},
 'total_vector_count': 100}

## Creating Vector Store and Querying

In [11]:
from langchain.schema.vectorstore import VectorStoreRetriever
from langchain.callbacks.manager import (
        AsyncCallbackManagerForRetrieverRun,
        CallbackManagerForRetrieverRun,
    )
from typing import (
    TYPE_CHECKING,
    Any,
    Callable,
    ClassVar,
    Collection,
    Dict,
    Iterable,
    List,
    Optional,
    Tuple,
    Type,
    TypeVar,
)
from langchain.schema.document import Document

class PineconeVectorStoreRetriever(VectorStoreRetriever):

     def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        if self.search_type == "similarity":
            docs = self.vectorstore.similarity_search(query, **self.search_kwargs)
        elif self.search_type == "similarity_score_threshold":
            docs_and_similarities = (
                self.vectorstore.similarity_search_with_relevance_scores(
                    query, **self.search_kwargs
                )
            )
            for doc, score in docs_and_similarities:
                doc.metadata = {**doc.metadata, **{"score": 1-score}}
            docs = [doc for doc, _ in docs_and_similarities]
        elif self.search_type == "mmr":
            docs = self.vectorstore.max_marginal_relevance_search(
                query, **self.search_kwargs
            )
        else:
            raise ValueError(f"search_type of {self.search_type} not allowed.")
        return docs 

In [12]:
from langchain.vectorstores import Pinecone

class PineconeModified(Pinecone):
    
    def as_retriever(self, **kwargs: Any) -> VectorStoreRetriever:
        """Return VectorStoreRetriever initialized from this VectorStore.

        Args:
            search_type (Optional[str]): Defines the type of search that
                the Retriever should perform.
                Can be "similarity" (default), "mmr", or
                "similarity_score_threshold".
            search_kwargs (Optional[Dict]): Keyword arguments to pass to the
                search function. Can include things like:
                    k: Amount of documents to return (Default: 4)
                    score_threshold: Minimum relevance threshold
                        for similarity_score_threshold
                    fetch_k: Amount of documents to pass to MMR algorithm (Default: 20)
                    lambda_mult: Diversity of results returned by MMR;
                        1 for minimum diversity and 0 for maximum. (Default: 0.5)
                    filter: Filter by document metadata

        Returns:
            VectorStoreRetriever: Retriever class for VectorStore.

        Examples:

        .. code-block:: python

            # Retrieve more documents with higher diversity
            # Useful if your dataset has many similar documents
            docsearch.as_retriever(
                search_type="mmr",
                search_kwargs={'k': 6, 'lambda_mult': 0.25}
            )

            # Fetch more documents for the MMR algorithm to consider
            # But only return the top 5
            docsearch.as_retriever(
                search_type="mmr",
                search_kwargs={'k': 5, 'fetch_k': 50}
            )

            # Only retrieve documents that have a relevance score
            # Above a certain threshold
            docsearch.as_retriever(
                search_type="similarity_score_threshold",
                search_kwargs={'score_threshold': 0.8}
            )

            # Only get the single most similar document from the dataset
            docsearch.as_retriever(search_kwargs={'k': 1})

            # Use a filter to only retrieve documents from a specific paper
            docsearch.as_retriever(
                search_kwargs={'filter': {'paper_title':'GPT-4 Technical Report'}}
            )
        """
        tags = kwargs.pop("tags", None) or []
        tags.extend(self._get_retriever_tags())

        return PineconeVectorStoreRetriever(vectorstore=self, **kwargs, tags=tags)

In [35]:
text_field = "description"

# switch back to normal index for langchain
index = pinecone.Index(index_name)

vectorstore = PineconeModified(
    index, embed, text_field
)
     

/Users/jonathan/miniforge3/envs/ds/lib/python3.8/site-packages/langchain/vectorstores/pinecone.py:59: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [36]:
query = "adventure story"

vectorstore.similarity_search_with_score(
    query, 
    k=3,
    filter={'category': 'popular'}
)



[(Document(page_content='Born the runt of his litter and gambled away to a rusty old river man, the Newfoundland pup Seaman doesn’t imagine his life will be marked by any kind of glory. But when he meets Captain Meriwether Lewis, Seaman finds himself on a path that will make history. Lewis is setting off on his landmark search for the Northwest Passage, and he takes Seaman along.\xa0Sharing the curiosity and spirit of his new master, the intrepid dog proves himself a valuable companion at every turn. \xa0\xa0\xa0\xa0 Part history, part science—and all adventure—this is the thrilling tale of America’s greatest journey of discovery.<br /><b>\n  <i>\xa0\xa0\xa0\xa0 Includes an author’s note and a reader<b><i>’</i></b>s guide.</i>\n</b>', metadata={'category': 'popular', 'title': "The Captain's Dog: My Journey with the Lewis and Clark Tribe"}),
  0.695170879),
 (Document(page_content='You and Carlos are on an expedition to the world\'s highest mountains in the remote kingdom of Nepal to se

In [22]:
query = "history"

vectorstore.similarity_search_with_score(
    query, 
    k=3,
    filter={'user_id' : '1' , 'category': 'recommended'}
)
     

[(Document(page_content='"The only work that treats Ku Kluxism for the entire period of it\'s existence. . . . <i>the </i>authoritative work on the period. <i>Hooded Americanism</i> is exhaustive in its rich detail and its use of primary materials to paint the picture of a century of terror. It is comprehensive, since it treats the entire period, and enjoys the perspective that the long view provides. It is timely, since it emphasizes the undeniable persistence of terrorism in American life."—John Hope Franklin', metadata={'category': 'recommended', 'title': 'Hooded Americanism: The History of the Ku Klux Klan, 3rd ed.', 'user_id': '1'}),
  0.781868577),
 (Document(page_content="This excellent resource takes full advantage of on-line information and students' familiarity with it. Reflecting the author's dual focus on environmental regulations and the policy considerations that shape them.", metadata={'category': 'recommended', 'title': 'Environmental Regulation: Statutory Supplement an

## Tools definition


In [23]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.schema.vectorstore import VectorStore
from langchain.chains.query_constructor.ir import StructuredQuery, Visitor
from langchain.schema.language_model import BaseLanguageModel
from langchain.retrievers.self_query.pinecone import PineconeTranslator
from langchain.chains.query_constructor.base import load_query_constructor_runnable

from typing import Any, Dict, List, Optional, Sequence, Tuple, Type, Union

In [24]:
def _get_builtin_translator(vectorstore: VectorStore) -> Visitor:
    """Get the translator class corresponding to the vector store class."""
    BUILTIN_TRANSLATORS: Dict[Type[VectorStore], Type[Visitor]] = {
        PineconeModified: PineconeTranslator,
    }
    
    if vectorstore.__class__ in BUILTIN_TRANSLATORS:
        return BUILTIN_TRANSLATORS[vectorstore.__class__]()
    else:
        raise ValueError(
            f"Self query retriever with Vector Store type {vectorstore.__class__}"
            f" not supported."
        )

class PineconeSelfQueryRetriever(SelfQueryRetriever):

    @classmethod
    def from_llm(
        cls,
        llm: BaseLanguageModel,
        vectorstore: VectorStore,
        document_contents: str,
        metadata_field_info: Sequence[Union[AttributeInfo, dict]],
        structured_query_translator: Optional[Visitor] = None,
        chain_kwargs: Optional[Dict] = None,
        enable_limit: bool = False,
        use_original_query: bool = False,
        **kwargs: Any,
    ) -> "SelfQueryRetriever":
        if structured_query_translator is None:
            structured_query_translator = _get_builtin_translator(vectorstore)
        chain_kwargs = chain_kwargs or {}

        if "allowed_comparators" not in chain_kwargs:
            chain_kwargs[
                "allowed_comparators"
            ] = structured_query_translator.allowed_comparators
        if "allowed_operators" not in chain_kwargs:
            chain_kwargs[
                "allowed_operators"
            ] = structured_query_translator.allowed_operators
        query_constructor = load_query_constructor_runnable(
            llm,
            document_contents,
            metadata_field_info,
            enable_limit=enable_limit,
            **chain_kwargs,
        )
        return cls(
            query_constructor=query_constructor,
            vectorstore=vectorstore,
            use_original_query=use_original_query,
            structured_query_translator=structured_query_translator,
            **kwargs,
        )

In [25]:
# chat completion llm

llm = ChatOpenAI(
    model_name='gpt-4-1106-preview',
    temperature=0.0
)

In [26]:
# conversational memory

conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

In [27]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

### Tool 1 - Generic Recommendation

In [28]:
metadata_field_info=[
    AttributeInfo(
        name="user_id",
        description="The user ID of the book recommendation",
        type="string",
    ),
    AttributeInfo(
        name="category",
        description="The type of entry (popular or recommended)",
        type="string or list[string]",
    ),
    AttributeInfo(
        name="title",
        description="The title of the book",
        type="string",
    ),
    AttributeInfo(
        name="description",
        description="The description of the book",
        type="float"
    ),
]
document_content_description = "The description of the book"

pineconeSelfQueryRetriever = PineconeSelfQueryRetriever.from_llm(llm, vectorstore, document_content_description, metadata_field_info, verbose=True)

In [29]:
## USER ID IS SPECIFICALLY DETERMINED IN FRONTEND

In [30]:
# retrieval qa chain

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=pineconeSelfQueryRetriever
)

In [31]:
%%time

qa.run("I am user 1 and recommend some books for me")

CPU times: user 32.9 ms, sys: 6.16 ms, total: 39.1 ms
Wall time: 20 s


'Based on the context provided, it seems you might be interested in scholarly works that delve into historical, social, and political themes. Here are some recommendations that align with the subjects mentioned:\n\n1. "Hooded Americanism: The History of the Ku Klux Klan" – This book is considered an authoritative work on the history of the Ku Klux Klan, covering the entire period of its existence. It provides a detailed examination of the group\'s impact on American society and the persistent issue of terrorism in American life.\n\n2. "Vision and Prophecy in Amos" by John D. W. Watts – If you\'re interested in biblical studies, particularly prophetic literature, this book is a classic in the field. It offers a form-critical analysis of the visions in the Book of Amos and includes new material that re-examines the text in the context of the Book of the Twelve.\n\n3. Any book on environmental regulations and policy – While a specific title isn\'t provided in the context, you might look f

#### Optimized version

In [32]:
%%time

generic_qa = pineconeSelfQueryRetriever | format_docs

generic_qa.invoke("I am user 1 and recommend some books for me")

CPU times: user 27.9 ms, sys: 3.23 ms, total: 31.2 ms
Wall time: 2.35 s


'First published in 1991. Routledge is an imprint of Taylor &amp; Francis, an informa company.\n\nThis excellent resource takes full advantage of on-line information and students\' familiarity with it. Reflecting the author\'s dual focus on environmental regulations and the policy considerations that shape them.\n\n"The only work that treats Ku Kluxism for the entire period of it\'s existence. . . . <i>the </i>authoritative work on the period. <i>Hooded Americanism</i> is exhaustive in its rich detail and its use of primary materials to paint the picture of a century of terror. It is comprehensive, since it treats the entire period, and enjoys the perspective that the long view provides. It is timely, since it emphasizes the undeniable persistence of terrorism in American life."—John Hope Franklin\n\nIn 1955 John D. W. Watts presented the faculty lectures at the Baptist Theological Seminary in Ruschlikon. Amos and prophetic studies have not been the same since. Three years later, in 19

### Tool 2 - Popular Recommendation

In [29]:
# retrieval qa chain

popular_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_rerank",
    retriever=vectorstore.as_retriever(
        search_kwargs={'k' : 5, 
                       'filter': {'category': 'popular'}}),
    # return_source_documents = True
)

In [30]:
popular_qa.run("Recommend me books about history")

/Users/jonathan/miniforge3/envs/ds/lib/python3.8/site-packages/langchain/chains/llm.py:349: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


"The Autobiography by Lincoln Steffens might be of interest to you as it covers historical events and the author's experiences with political figures and significant societal changes."

#### Optimized version

In [33]:
pinecone_retriever = vectorstore.as_retriever(
                search_kwargs={'k' : 5, 
                                'filter': {'category': 'popular'}})

popular_qa = pinecone_retriever | format_docs

popular_qa.invoke("Recommend me books about history")

'This bracing novel introduces vivid African-American characters into the traditional territory of James Michener and Larry McMurtry, as it tells the powerful story of two black families in 1920\'s Texas entwined in a triumphant saga of love and loss, greed, obsession, and revenge.\n\nThe most nostalgic and reflective of Evelyn Waugh\'s novels, <i>Brideshead Revisited</i> looks back to the golden age before the Second World War. It tells the story of Charles Ryder\'s infatuation with the Marchmains and the rapidly-disappearing world of privilege they inhabit. Enchanted first by Sebastian at Oxford, then by his doomed Catholic family, in particular his remote sister, Julia, Charles comes finally to recognize only his spiritual and social distance from them.\n\nRediscover a man Americans turned to not only for news but for humor &amp; wisdom. Growing up in Sacramento, Steffens (1866-1936) was an editor at the NY Evening Post, later at McClure’s Magazine. As popular as he was cantankerous

### Tool 3 - Specific Recommendation

In [34]:
recommended_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_rerank",
    retriever=vectorstore.as_retriever(
        search_kwargs={'k' : 5, 
                       'filter': {'user_id' : '1', 'category': 'recommended'}}),
    # return_source_documents = True
)

In [35]:
recommended_qa.invoke("Recommend me books about history")

/Users/jonathan/miniforge3/envs/ds/lib/python3.8/site-packages/langchain/chains/llm.py:349: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


{'query': 'Recommend me books about history',
 'result': '"Hooded Americanism" is a book that might interest you as it covers the history of the Ku Klux Klan over a century, providing detailed information and analysis based on primary materials.'}

#### Optimized Version

In [52]:
def format_docs_title(docs):
    return "\n\n".join([f"{i+1}. {d.metadata['title']} : {d.page_content}" for i,d in enumerate(docs)])

In [55]:
pineconeRetreiver = vectorstore.as_retriever(
        search_kwargs={'k' : 5, 
                       'filter': {'user_id' : '1', 'category': 'recommended'}})

recommended_qa = pineconeRetreiver | format_docs_title

recommended_qa.invoke("history books")

'1. Hooded Americanism: The History of the Ku Klux Klan, 3rd ed. : "The only work that treats Ku Kluxism for the entire period of it\'s existence. . . . <i>the </i>authoritative work on the period. <i>Hooded Americanism</i> is exhaustive in its rich detail and its use of primary materials to paint the picture of a century of terror. It is comprehensive, since it treats the entire period, and enjoys the perspective that the long view provides. It is timely, since it emphasizes the undeniable persistence of terrorism in American life."—John Hope Franklin\n\n2. Informal Economy: A Research G : First published in 1991. Routledge is an imprint of Taylor &amp; Francis, an informa company.\n\n3. Environmental Regulation: Statutory Supplement and Guide to Internet Resources, 2002 Edition : This excellent resource takes full advantage of on-line information and students\' familiarity with it. Reflecting the author\'s dual focus on environmental regulations and the policy considerations that sha

In [56]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate
from operator import itemgetter

# Check if the recommended books is match the user's query
chain = (
    {"recommended_books": recommended_qa, "query": RunnablePassthrough()}
    | ChatPromptTemplate.from_template(
        """
        Check if the document recommends a book. Say "yes" or "no".

        Recommended Books: 
        {recommended_books}

        Classification:"""
    )
    | llm
    | StrOutputParser()
)

# If yes
recommended_chain = (
    {"recommended_books" : RunnablePassthrough()}
    | PromptTemplate.from_template(
        """
        You are an expert in recommended books. \
        Give the user book recommendation books using below information. \
        Always start with "I have some books recommendation that is tailored to your taste. \
        
        Recommended Books: 
        {recommended_books}
        """
    )
    | llm
)

# If no
popular_chain = (
    {"recommended_books": popular_qa, "question": RunnablePassthrough()}
    | PromptTemplate.from_template(
        """
        You are an expert in recommended books. \
        Give the user book recommendation books using below information. \
        Always start with "I have some popular books that I can recommend for you. \
        
        Recommended Books: 
        {recommended_books}
        """
    )
    | llm
)

from langchain.schema.runnable import RunnableBranch

def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

full_chain = (
    {
        "topic": (
            {"recommended_books": recommended_qa , "query": RunnablePassthrough()}
            | ChatPromptTemplate.from_template(
                """
                Check if the document recommends a book. Say "yes" or "no".

                Recommended Books: 
                {recommended_books}

                Classification:"""
            )
            | llm
            | StrOutputParser()
            ), 
        "query": RunnablePassthrough()
    }
    | RunnableBranch(
        (lambda x: "yes" in x["topic"].lower() or "Yes" in x["topic"].lower(), (lambda x :  x['query']) | recommended_qa),
        (lambda x: "no" in x["topic"].lower() or "No" in x["topic"].lower(), (lambda x :  x['query']) | popular_chain),
        (lambda x :  x['query']) | popular_chain
        )
    | StrOutputParser()
)


In [57]:
full_chain.invoke("I am a user 1 and recommend some books about romance for me")

'I have some popular books that I can recommend for you:\n\n1. If you enjoy historical romance with a festive touch, you might like a novel where "a wary widow is befriended by a charming duke who\'s determined not only to win her trust, but convince her that she is the one woman for him at Christmas and for all seasons." A book that fits this description is "Once Upon a Christmas Eve" by Elizabeth Hoyt, part of her \'Maiden Lane\' series.\n\n2. For those interested in medieval literature and its connections to classical works, a book that "seeks to define the medieval literary conventions governing allusions to certain Ovidian and Virgilian tales of love in the works of Boccaccio, Machaut, Froissart, and Chaucer" would be a scholarly read. "Tradition and Innovation in Chaucer" by A. C. Spearing may offer insights into these topics.\n\n3. If you\'re fascinated by the history of printing and typography, a book discussing "<i>The Dolphin and The Fleuron</i>," which established an interna

In [58]:
async for chunk in full_chain.astream_log(
    "I am a user 1 and recommend some books about romance for me"
):
    print("-" * 80)
    print(chunk)

--------------------------------------------------------------------------------
RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': '55334475-78e4-4d0d-baeb-e3df16f4d4fa',
            'logs': {},
            'streamed_output': []}})
--------------------------------------------------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/RunnableParallel',
  'value': {'end_time': None,
            'final_output': None,
            'id': '560ccf1d-811d-439e-935c-031e0e9805f6',
            'metadata': {},
            'name': 'RunnableParallel',
            'start_time': '2023-11-13T01:00:20.986',
            'streamed_output_str': [],
            'tags': ['seq:step:1'],
            'type': 'chain'}})
--------------------------------------------------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/RunnableSequence',
  'value': {'end_time': None,
            'final_output': None,
            'id': '045b

/Users/jonathan/miniforge3/envs/ds/lib/python3.8/site-packages/langchain/chains/llm.py:371: UserWarning: The aapply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


--------------------------------------------------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/ChatOpenAI/final_output',
  'value': {'generations': [[{'generation_info': {'finish_reason': 'stop'},
                              'message': AIMessage(content='This document does not answer the question\nScore: 0'),
                              'text': 'This document does not answer the '
                                      'question\n'
                                      'Score: 0',
                              'type': 'ChatGeneration'}]],
            'llm_output': {'model_name': 'gpt-4-1106-preview',
                           'token_usage': {'completion_tokens': 12,
                                           'prompt_tokens': 338,
                                           'total_tokens': 350}},
            'run': None}},
 {'op': 'add',
  'path': '/logs/ChatOpenAI/end_time',
  'value': '2023-11-13T01:00:23.804'})
-----------------------------------------

In [37]:
full_chain.invoke("I am user 1 and recommend me books about history")

/Users/jonathan/miniforge3/envs/ds/lib/python3.8/site-packages/langchain/chains/llm.py:349: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/jonathan/miniforge3/envs/ds/lib/python3.8/site-packages/langchain/chains/llm.py:349: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


'I have some book recommendations that are tailored to your taste. Since you\'re interested in history, "Hooded Americanism" is a book that might interest you as it covers the history of the Ku Klux Klan over a century, providing detailed insights from primary materials.'

In [59]:
full_chain.invoke("history")

/Users/jonathan/miniforge3/envs/ds/lib/python3.8/site-packages/langchain/chains/llm.py:349: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


'I have some popular books that I can recommend for you:\n\n1. For a deep dive into medieval warfare and its impact on English and French history, you might be interested in a book that provides the "First large-scale study of Crécy and its context, bringing out its full significance in English and French history." This book promises to offer a comprehensive analysis of the Battle of Crécy and its historical implications, making it a valuable read for enthusiasts of military history and medieval studies.\n\n2. If you\'re fascinated by ancient civilizations and their cultural legacies, consider a book that is "a timely work which seeks to place Egypt in its true historical context." This book explores the African origins of the ancient Egyptians, their advancements in arts and sciences, and their social systems. It\'s an enlightening read that challenges conventional narratives and is recommended for those interested in Egyptology and African history.\n\n3. For those intrigued by the co

In [58]:
full_chain.invoke("history books")

'1. Hooded Americanism: The History of the Ku Klux Klan, 3rd ed. : "The only work that treats Ku Kluxism for the entire period of it\'s existence. . . . <i>the </i>authoritative work on the period. <i>Hooded Americanism</i> is exhaustive in its rich detail and its use of primary materials to paint the picture of a century of terror. It is comprehensive, since it treats the entire period, and enjoys the perspective that the long view provides. It is timely, since it emphasizes the undeniable persistence of terrorism in American life."—John Hope Franklin\n\n2. Informal Economy: A Research G : First published in 1991. Routledge is an imprint of Taylor &amp; Francis, an informa company.\n\n3. Environmental Regulation: Statutory Supplement and Guide to Internet Resources, 2002 Edition : This excellent resource takes full advantage of on-line information and students\' familiarity with it. Reflecting the author\'s dual focus on environmental regulations and the policy considerations that sha

### Tool Consolidation

In [46]:
from langchain.agents import Tool

tools = [
    Tool(
        name='Generic Recommendation',
        func=qa.run,
        description=(
            'use this tool when the user asking for book recommendation without any specific preference'
        )
    ),
    Tool(
        name='Specific Recommendation',
        func=full_chain.invoke,
        description=(
            'use this tool when the user asking for book recommendation with a specific preference (genre, theme, etc.)'
        )
    ),
    Tool(
        name='Popular Recommendation',
        func=popular_qa.invoke,
        description=(
            'use this tool when the user asking for popular book recommendation without any specific preference'
        )
    ),
    Tool(
        name='Generic Prompt',
        func=llm.invoke,
        description=(
            'use this tool when the user asking or talk about general question'
        )
    ),
]

## Conversation Agent

In [47]:
# conversational memory

conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

In [48]:
from langchain.agents import initialize_agent

agent = initialize_agent(
    # agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory
)

In [49]:
agent("Recommend me some books about history")



> Entering new AgentExecutor chain...
The user is asking for book recommendations with a specific preference, which is the genre of history.
Action: Specific Recommendation
Action Input: history books
Observation: "The only work that treats Ku Kluxism for the entire period of it's existence. . . . <i>the </i>authoritative work on the period. <i>Hooded Americanism</i> is exhaustive in its rich detail and its use of primary materials to paint the picture of a century of terror. It is comprehensive, since it treats the entire period, and enjoys the perspective that the long view provides. It is timely, since it emphasizes the undeniable persistence of terrorism in American life."—John Hope Franklin

First published in 1991. Routledge is an imprint of Taylor &amp; Francis, an informa company.

This excellent resource takes full advantage of on-line information and students' familiarity with it. Reflecting the author's dual focus on environmental regulations and the policy considerations 

{'input': 'Recommend me some books about history',
 'chat_history': [],
 'output': 'Here are some history books that might interest you:\n\n1. "Hooded Americanism: The History of the Ku Klux Klan" – This book is an authoritative work on the history of the Ku Klux Klan, providing a detailed account of its existence over a century and its impact on American life.\n\n2. An unnamed book that seems to focus on environmental regulations and policy considerations, reflecting the author\'s expertise in the field. (The specific title and author were not provided in the observation.)\n\n3. "Vision and Prophecy in Amos" by John D. W. Watts – This is a form-critical analysis of the visions of Amos, offering insights into biblical prophecy and its literary context within the Book of the Twelve.\n\n4. "In Amma\'s Healing Room: Gender and Vernacular Islam in South India" by Joyce Burkhalter Flueckiger – This book provides a study of a female Muslim spiritual healer in Hyderabad, India, exploring the 

In [65]:
agent("Great, I love it")



> Entering new AgentExecutor chain...
The user has expressed a positive sentiment but hasn't asked a question or provided a context for a specific action.
Action: Generic Prompt
Action Input: Could you tell me more about what you love? It would help me assist you better.
Observation: content="As an artificial intelligence, I don't have personal feelings or preferences, so I don't experience love or affection in the way humans do. However, I am programmed to enjoy assisting and providing information to users like you. My purpose is to help answer your questions, provide accurate information, and facilitate a positive experience during our interactions.\n\nIf you have specific topics or areas you're interested in, feel free to ask, and I'll do my best to provide you with the information you're looking for!"
Thought:The user's response seems to be a misunderstanding. They might have been expressing enthusiasm about something else and not directly engaging with me. It's possible they wer

{'input': 'Great, I love it',
 'chat_history': [HumanMessage(content='Recommend me some books about history'),
  AIMessage(content='Here are some history book recommendations for you:\n\n1. A detailed study of the Battle of Crécy, exploring its significance in English and French history, particularly during the Hundred Years\' War.\n2. An exploration of the African origin of the ancient Egyptians, their achievements, and their contributions to writing, arts, sciences, and social systems.\n3. A novel set in 1920s Texas that brings African-American history to life through a narrative filled with love, loss, greed, and revenge.\n4. A comprehensive index on "The Dolphin and The Fleuron," providing a detailed listing of the contents of these influential publications in the history of printing and typography.\n5. A historical analysis of the visual language used by Late Preclassic rulers in Chiapas, Mexico, to express political and supernatural authority, offering insights into the iconograp